In [1]:
!pip install langchain-text-splitters
!pip install langchain-huggingface
!pip install langchain-faiss-cpu

ERROR: Could not find a version that satisfies the requirement langchain-faiss-cpu (from versions: none)
ERROR: No matching distribution found for langchain-faiss-cpu


In [1]:
# IMPORTAÇÕES

import os  # O 'os' serve para interagir com o sistema operacional (ex: ver se um arquivo existe)
import json  # Nossos dados estão em .jsonl, então precisamos do 'json' para conseguir lê-los
import re  # (Regex) É o nosso "faxineiro" de texto, usado para tirar sujeira e caracteres estranhos
from llama_cpp import Llama  # É a biblioteca que carrega o "Cérebro" (o LLM Phi-3)

# Bibliotecas do Buscador (RAG)
# Essas ferramentas são do LangChain e servem para construir nossa "Biblioteca" (o RAG)
from langchain_text_splitters import RecursiveCharacterTextSplitter # A "tesoura" que corta textos grandes em pedaços menores
from langchain_huggingface import HuggingFaceEmbeddings # O "tradutor" que transforma os textos em números (vetores)
from langchain_community.vectorstores import FAISS # O "Google" particular, nosso banco de dados vetorial super-rápido

# Ferramentas Padrão (A "COLA" do LangChain)
# São as peças que vão conectar tudo
from langchain_core.prompts import PromptTemplate # O "roteiro" que diz ao LLM como ele deve se comportar
from langchain_core.runnables import RunnablePassthrough # Uma "peça" que deixa a pergunta do usuário passar direto
from langchain_core.output_parsers import StrOutputParser  # Garante que a resposta final seja só o texto, e nada mais

print("--- Inicializando a Pipeline do Chatbot ---")

# FUNÇÃO PARA LER OS ARQUIVOS .JSONL (O "BIBLIOTECÁRIO") 
# Precisamos de uma "receita" (função) para ler todos os nossos arquivos de texto.
def carregar_textos_dos_jsonl(arquivos_jsonl):
    """
    Esta função abre uma lista de arquivos .jsonl, lê linha por linha,
    e puxa o texto de dentro de cada uma.
    """
    textos_completos = []  # Uma "caixa" vazia para guardar todos os textos que encontrarmos
    print(f"Carregando textos de {len(arquivos_jsonl)} arquivos...")
    
    # Vamos olhar cada arquivo da nossa lista, um por um
    for arquivo in arquivos_jsonl:
        # Primeiro, checamos se o arquivo realmente existe no lugar certo
        if not os.path.exists(arquivo):
            print(f"AVISO: Arquivo {arquivo} não encontrado. Pulando.")
            continue  # Vai para o próximo arquivo da lista
        
        # O 'try...except' é um "cinto de segurança". Se der erro ao ler, o programa não quebra.
        try:
            # Abrimos o arquivo (usando 'utf-8' para ler acentos e 'ç' corretamente)
            with open(arquivo, 'r', encoding='utf-8') as f:
                # Como é .jsonl, lemos linha por linha
                for linha in f:
                    try:
                        # Se a linha estiver vazia, pulamos
                        if not linha.strip(): continue
                        
                        # Transformamos o texto da linha (JSON) em um objeto Python (um dicionário)
                        data = json.loads(linha)
                        
                        # Agora, procuramos o texto!
                        if isinstance(data, dict):
                            # Às vezes o texto está na chave "text"
                            if "text" in data:
                                textos_completos.append(data["text"])
                            # Às vezes está na chave "page_content" (comum no LangChain)
                            elif "page_content" in data:
                                textos_completos.append(data["page_content"])
                        # E se o JSON for só o texto puro?
                        elif isinstance(data, str):
                            textos_completos.append(data)
                            
                    except json.JSONDecodeError: pass  # Se a linha não for um JSON válido, a gente só ignora
                    except Exception as e_linha: print(f"Aviso: Erro ao processar linha no arquivo {arquivo}: {e_linha}")
                        
        except Exception as e_arquivo:
            print(f"Erro GERAL ao ler o arquivo {arquivo}: {e_arquivo}")
            
    print(f"Textos carregados. Total de {len(textos_completos)} trechos de texto.")
    
    # Alerta importante! Se não achamos NADA, o chatbot não vai funcionar.
    if len(textos_completos) == 0:
        print("ALERTA CRÍTICO: Nenhum texto foi carregado.")
        
    # No final, juntamos todos os pedacinhos de texto em um "textão" gigante
    return "\n".join(textos_completos)

#CARREGAR O "GERADOR" (LLM - O "CÉREBRO")
# Agora vamos carregar o modelo de linguagem que vai "pensar" e escrever as respostas.

# O nome do arquivo .gguf do modelo.
model_name = "Phi-3-mini-4k-instruct-Q4_K_S.gguf"
# Pegamos o caminho completo (ex: C:/Usuario/Projeto/modelo.gguf)
model_path_absolute = os.path.abspath(model_name)

# Verificação de segurança: O arquivo do modelo está onde deveria?
if not os.path.exists(model_path_absolute):
    # Se não encontrar, o programa para aqui.
    print(f"ERRO CRÍTICO: Não encontrou o modelo em {model_path_absolute}")
else:
    # Se encontrou, continuamos
    print(f"SUCESSO: Encontrado o modelo em: {model_path_absolute}")
    print("Carregando o Gerador (LLM Phi-3)... Isso pode demorar um pouco.")
    
    # Esta é a linha que "chama" o cérebro para a memória
    llm = Llama(
        model_path=model_path_absolute,  # Onde está o arquivo
        n_gpu_layers=50,  # 50 camadas do modelo na Placa de Vídeo (GPU)!
                          # Isso deixa o chatbot MUITO mais rápido. (Se der erro, tente 0 ou -1)
        n_ctx=4096,       # A "memória de curto prazo" do modelo. O tamanho do "papel" que ele tem
                          # para ler o contexto e escrever a resposta (em tokens).
        verbose=False     # Para ele não ficar "falando" (imprimindo logs) o tempo todo.
    )
    print("Gerador (LLM) carregado com sucesso!")

    # "BUSCADOR" (A "BIBLIOTECA" RAG) 
    # Agora vamos pegar nossos documentos, "quebrar" e "indexar" eles
    # para que o chatbot possa fazer buscas rápidas.
    print("\nCriando o Buscador (RAG)...")
    
    lista_de_documentos = [
        "Estágio_output (1).jsonl",
        "Estatuto - Fevereiro 2025__output (1).jsonl",
        "PPCBCC2019_output (1).jsonl",
        "Regimento Interno dos Campi_output (1).jsonl",
        "REGIMENTO_GERAL_FEVEREIRO_DE_2025_output (1).jsonl",
        "RegulamentoCursosGraduação_output (1).jsonl",
        "Ciencia-da-computacao-01-2025_com_sala_MkIII_output (2).jsonl"
    ]
    
    # Usamos nossa função "Bibliotecário" para ler tudo e juntar no "textão"
    textos_da_faculdade = carregar_textos_dos_jsonl(lista_de_documentos)

    # Se o textão estiver vazio, não temos o que fazer.
    if not textos_da_faculdade:
        print("ERRO: Nenhum texto foi carregado. A pipeline não pode ser criada.")
    else:
        # "Tesoura" (Splitter)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,  # Cortar os textos em pedaços (chunks) de 1000 caracteres
            chunk_overlap=200 # Cada pedaço novo vai ter 200 caracteres do pedaço anterior
                              # (Isso evita cortar uma frase/ideia importante no meio)
        )
        # Cortando o "textão" em uma lista de 'chunks'
        chunks = text_splitter.split_text(textos_da_faculdade)
        
        print(f"Textos divididos em {len(chunks)} 'chunks'.")
        
        # "Tradutor" (Embeddings)
        print("Carregando modelo de embedding (sentence-transformer)...")
        embeddings_model = HuggingFaceEmbeddings(
            model_name="paraphrase-multilingual-MiniLM-L12-v2", 
            model_kwargs={'device': 'cpu'}  # O tradutor é leve, pode rodar na CPU
        )
        
        #  "Índice" (Vector Store - FAISS)
        # Este é o passo mais importante do RAG.
        # Ele pega os 'chunks', usa o 'tradutor' para transformar em números (vetores),
        # e armazena tudo no FAISS, que é um índice de busca ultra-rápido.
        print("Criando o índice vetorial (FAISS)... Isso pode levar um minuto.")
        vector_store = FAISS.from_texts(chunks, embeddings_model)
        
        #  O "Buscador" (Retriever)
        # Agora criamos o "Buscador" oficial, que usa o índice FAISS
        
        # MUDAMOS DE 'k=3' PARA 'k=2'
        # Isso reduz o "ruído" (lixo) enviado para o LLM.
        # k=2 significa: "Quando alguém fizer uma pergunta, me traga os 2 chunks
        # mais parecidos com ela."
        retriever = vector_store.as_retriever(search_kwargs={'k': 2})
        
        print("Buscador (RAG) criado com sucesso (k=2)!")

        # --- 5. MONTAR A PIPELINE (A "LINHA DE MONTAGEM") ---
        # Agora vamos conectar o "Buscador" (RAG) com o "Gerador" (LLM).
        print("\nMontando a pipeline final...")

        #  "ROTEIRO"
        # Damos ao Phi-3 permissão para filtrar o lixo e encontrar a resposta certa.
        # Este roteiro é a "instrução" que o LLM recebe ANTES de ver a pergunta.
        prompt_template = """<|system|>
Você é um assistente prestativo do IFNMG.
Sua tarefa é responder a pergunta do usuário.
Use os trechos de contexto abaixo para encontrar a resposta mais precisa.
O contexto pode conter informações misturadas; seu trabalho é encontrar a informação correta
e responder de forma direta e concisa.
Responda APENAS com base no contexto fornecido.<|end|>
<|user|>
CONTEXTO:
{context}

PERGUNTA:
{question}<|end|>
<|assistant|>
"""
        # {context} e {question} são "buracos" que vamos preencher automaticamente.
        # <|assistant|> no final já deixa "no ponto" para o LLM começar a resposta.
        
        # Transformamos o texto do roteiro em um objeto "PromptTemplate" do LangChain
        prompt = PromptTemplate(
            template=prompt_template,
            input_variables=["context", "question"] # Os "buracos" que ele precisa preencher
        )
        
        # FUNÇÃO DE LIMPEZA DO CONTEXTO (O "FAXINEIRO")
        # O RAG pode trazer textos com caracteres estranhos (ex: ícones 0uf000)
        # Esta função vai limpar esse lixo antes de mandar para o LLM.
        def formatar_e_limpar_contexto(docs: list) -> str:
            textos_limpos = []
            for doc in docs:
                texto_sujo = doc.page_content # Pegamos o texto do chunk
                # 1ª Faxina: Remover caracteres estranhos (tipo  UF000)
                texto_limpo = re.sub(r'[\uf000-\uf0ff]', ' ', texto_sujo)
                # 2ª Faxina: Substituir múltiplos espaços/quebras de linha por um espaço só
                texto_limpo = re.sub(r'\s+', ' ', texto_limpo)
                textos_limpos.append(texto_limpo.strip())
            # Junta os (k=2) chunks limpos em um texto só, separados por uma linha
            return "\n".join(textos_limpos)

        # FUNÇÃO DE RESPOSTA (O "CHAMADOR" DO LLM) 
        # Fizemos isso porque o `Llama` do `llama_cpp` funciona um pouco
        # diferente do `Llama` padrão do LangChain.
        def llm_responder(prompt_como_objeto):
            # O LangChain vai entregar um "objeto" de prompt.
            # O `llama_cpp` só quer uma string de texto. Então a gente converte.
            prompt_como_string = prompt_como_objeto.to_string()
            
            # Mandamos o prompt completo (roteiro + contexto + pergunta) para o "Cérebro"
            output = llm(
                prompt_como_string,
                max_tokens=1024,      # A resposta pode ter no máximo 1024 tokens
                temperature=0.2,    # 0.0 = 100% focado, 1.0 = 100% criativo.
                                    # 0.2 é ótimo para "focar" na resposta do RAG.
                repeat_penalty=1.1  # Uma leve penalidade se ele começar a se repetir.
            )
            # A resposta do LLM vem dentro de um JSON. Pegamos só o texto.
            return output["choices"][0]["text"]

        # A PIPELINE FINAL (A "LINHA DE MONTAGEM")
        # ALangChain. O símbolo `|` (pipe)
        # significa: "o resultado do passo anterior vira a entrada do próximo".
        
        chain = (
            {
                # O processo começa aqui. Quando o usuário perguntar algo:
                # 1. A pergunta vai para o 'retriever', que acha os 2 chunks.
                # 2. Os 2 chunks são "pipeados" para a função 'formatar_e_limpar_contexto'.
                # O resultado limpo vira a variável "context".
                "context": retriever | formatar_e_limpar_contexto,
                
                # 3. A pergunta original do usuário passa direto (Passthrough)
                # e vira a variável "question".
                "question": RunnablePassthrough()
            }
            # 4. As variáveis "context" e "question" são encaixadas no "roteiro" (prompt)
            | prompt
            # 5. O roteiro completo é "pipeado" para o "Cérebro" (llm_responder)
            | llm_responder
            # 6. A resposta final do "Cérebro" é "pipeada" para o "Limpador de Saída",
            # que garante que teremos só o texto puro.
            | StrOutputParser()
        )

        print("\n CHATBOT PRONTO PARA USAR! (Versão V2) ")
        print("Rode a próxima célula para fazer perguntas.")

--- Inicializando a Pipeline do Chatbot ---
SUCESSO: Encontrado o modelo em: D:\ProjetoIntegrador\Phi-3-mini-4k-instruct-Q4_K_S.gguf
Carregando o Gerador (LLM Phi-3)... Isso pode demorar um pouco.
Gerador (LLM) carregado com sucesso!

Criando o Buscador (RAG)...
Carregando textos de 7 arquivos...
Textos carregados. Total de 8697 trechos de texto.
Textos divididos em 570 'chunks'.
Carregando modelo de embedding (sentence-transformer)...
Criando o índice vetorial (FAISS)... Isso pode levar um minuto.
Buscador (RAG) criado com sucesso (k=2)!

Montando a pipeline final...

 CHATBOT PRONTO PARA USAR! (Versão V2) 
Rode a próxima célula para fazer perguntas.


In [3]:
print("Buscando resposta...")

# --- FAÇA SUA PERGUNTA AQUI ---
pergunta = "Quais são os professores que podem ser orientadores do TCC?"
# ---

resposta = chain.invoke(pergunta)

print("\n--- Resposta ---")
print(resposta)

Buscando resposta...

--- Resposta ---
 Os professores que podem ser orientadores do TCC são os professores de Projeto de Trabalho de Conclusão de Curso (PTCC) no IFNMG - Campus Montes Claros.


In [4]:
print("Buscando resposta...")

# --- FAÇA SUA PERGUNTA AQUI ---
pergunta = "Qual é o tempo máximo permitido para que um estudante conclua o curso de Ciência da Computação?"
# ---

resposta = chain.invoke(pergunta)

print("\n--- Resposta ---")
print(resposta)

Buscando resposta...

--- Resposta ---
 O tempo máximo permitido para que um estudante conclua o curso de Ciência da Computação é 3280 horas.


In [5]:
print("Buscando resposta...")

# --- FAÇA SUA PERGUNTA AQUI ---
pergunta = "Como o aluno sabe se foi aprovado ou reprovado numa matéria?"
# ---

resposta = chain.invoke(pergunta)

print("\n--- Resposta ---")
print(resposta)

Buscando resposta...

--- Resposta ---
 O aluno sabe se foi aprovado ou reprovado numa matéria observando a obrigatoriedade da realização de, no mínimo, 03 (três) avaliações parciais no período letivo e sendo informado pelo professor sobre os resultados obtidos nos instrumentos de avaliação.


In [8]:
print("Buscando resposta...")

# --- FAÇA SUA PERGUNTA AQUI ---
pergunta = "Quantas horas e que tipo de atividades contam como Atividades Complementares no curso de Ciência da Computação?"
# ---

resposta = chain.invoke(pergunta)

print("\n--- Resposta ---")
print(resposta)

Buscando resposta...

--- Resposta ---
 As Atividades Complementares no curso de Ciência da Computação totalizam 160 horas.

(Note: The context does not specify the types of activities that count as "Complementary Activities," so only the number of hours is provided.)


In [10]:
print("Buscando resposta...")

# --- FAÇA SUA PERGUNTA AQUI ---
pergunta = " Em qual momento do curso a gente pode começar a fazer o estágio de Ciência da Computação?"
# ---

resposta = chain.invoke(pergunta)

print("\n--- Resposta ---")
print(resposta)

Buscando resposta...

--- Resposta ---
 O estágio de Ciência da Computação pode começar após o período integral do curso, uma vez que a matrícula é adotada e os estudantes já possuem conhecimento básico necessário.
